# The objective of this tutorial is to use the AppEEARS API to extract LST for two wildfires in Montana

#### Select code adapted from: https://git.earthdata.nasa.gov/projects/LPDUR/repos/appeears-api-getting-started/browse

### To extract wildfire polygons from Google Earth Engine, click the link below
### https://code.earthengine.google.com/cc8a5e7886d2a02719f87ce13bc7dc40

In [1]:
# Import packages 
import requests as r
import getpass, pprint, time, os, cgi, json # getpass needed for Earthdata login
import geopandas as gpd

In [2]:
# Set input directory, change working directory
inDir = '/home/marie/wildfire/LST'           # working directory
os.chdir(inDir)                                      # Change to working directory
api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable

In [3]:
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

Enter NASA Earthdata Login Username: ········
Enter NASA Earthdata Login Password: ········


#### The requests package will post your username and password. The token received will be used later to submit your request.

In [4]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
del user, password                                                           # Remove user and password information
token_response                                                               # Print response

{'token_type': 'Bearer',
 'token': 'x0ahhDEY3SPOPLMeCna-P5HQGTpjKldmRJvRMRswmNwmLuhd8jmxkOtf5quPaoGnVKqwaRzAVs27r7_tfzLnoQ',
 'expiration': '2022-10-20T01:29:40Z'}

#### Available products from appears
https://appeears.earthdatacloud.nasa.gov/products

In [5]:
# The get() method returns the value of the item with the specified key.
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service

In [8]:
products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by product name & version
products['ECO2LSTE.001']                                         # Print information for the product of interest ECO2LSTE.001 LST Product

{'Product': 'ECO2LSTE',
 'Platform': 'ECOSTRESS',
 'Description': 'Land Surface Temperature & Emissivity (LST&E)',
 'RasterType': 'Swath',
 'Resolution': '70m',
 'TemporalGranularity': 'ISS-dependent',
 'Version': '001',
 'Available': True,
 'DocLink': 'https://doi.org/10.5067/ECOSTRESS/ECO2LSTE.001',
 'Source': 'LP DAAC',
 'TemporalExtentStart': '2018-07-09',
 'TemporalExtentEnd': 'Present',
 'Deleted': False,
 'DOI': '10.5067/ECOSTRESS/ECO2LSTE.001',
 'ProductAndVersion': 'ECO2LSTE.001'}

#### I would like explore what layers are within the LST product

In [10]:
prods = ['ECO2LSTE.001']     # Start a list for products to be requested, beginning with ECO2LSTE.001
lst_response = r.get('{}product/{}'.format(api, prods[0])).json()  # Request layers for the only product (index 0) in the list: ECO2LSTE.001
list(lst_response.keys())

['SDS_Emis1',
 'SDS_Emis1_err',
 'SDS_Emis2',
 'SDS_Emis2_err',
 'SDS_Emis3',
 'SDS_Emis3_err',
 'SDS_Emis4',
 'SDS_Emis4_err',
 'SDS_Emis5',
 'SDS_Emis5_err',
 'SDS_EmisWB',
 'SDS_LST',
 'SDS_LST_err',
 'SDS_PWV',
 'SDS_QC']

#### I'm only interested in SDS_LST right now, so I will use that dictionary key to learn more about that layer from the lst_response variable

In [11]:
lst_response['SDS_LST']

{'AddOffset': 0.0,
 'Available': True,
 'DataType': 'uint16',
 'Description': 'Land Surface Temperature',
 'Dimensions': ['FakeDim0', 'FakeDim1'],
 'FillValue': 0.0,
 'Group': '',
 'IsQA': False,
 'Layer': 'SDS_LST',
 'OrigDataType': 'uint16',
 'OrigValidMax': 65535,
 'OrigValidMin': 7500,
 'QualityLayers': "['SDS_QC']",
 'QualityProductAndVersion': 'ECO2LSTE.001',
 'ScaleFactor': 0.02,
 'Units': 'K',
 'ValidMax': 1310.7,
 'ValidMin': 150.0,
 'XSize': 5400,
 'YSize': 5632}

In [12]:
# Adding QC layer
layers = [(prods[0],'SDS_LST'),(prods[0],'SDS_QC')]  # Create tupled list linking desired product with desired layers

#### Here we are created a dictionary list from our tupled list (the layers) that stores LST and LST QC. These will be used to insert into the json file when we submit our requests.

In [13]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

[{'layer': 'SDS_LST', 'product': 'ECO2LSTE.001'},
 {'layer': 'SDS_QC', 'product': 'ECO2LSTE.001'}]

# Submit Request

### Document this part

In [15]:
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

In [19]:
# czHorse = gpd.read_file('/home/marie/wildfire/LST/crazyHorseShapefile'.format(inDir + os.sep + 'Data' + os.sep)) # Read in shapefile as dataframe using geopandas
czHorse = gpd.read_file('crazyHorseShapefile.shp'.format(inDir + os.sep + 'Data' + os.sep)) # Read in shapefile as dataframe using geopandas
print(czHorse.head())                                                # Print first few lines of dataframe

  Comment Incid_Type  High_T  Mod_T  NoData_T BurnBndLon  Low_T  \
0    None   Wildfire     660    371      -970   -113.797    120   

         Ig_Date Map_Prog  dNBR_stdDv  ...               Event_ID  Perim_ID  \
0  1060498800000     MTBS       -9999  ...  MT4743511379720030810      None   

  BurnBndLat dNBR_offst BurnBndAc IncGreen_T  irwinID          Post_ID  \
0     47.435         15     11423       -150     None  504102720040925   

   Asmnt_Type                                           geometry  
0    Extended  POLYGON ((-113.86393 47.43105, -113.86337 47.4...  

[1 rows x 23 columns]


### Set projection of choice LEFT OFF HERE

In [ ]:
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections                                               # Print projections and information

In [ ]:
# Figure out how to insert the geographic projection here
projs = {}                                  # Create a dictionary with one entry, we're going to set the projection to geographic

### Convert to json

In [22]:
czHorseJ = czHorse.to_json() 
czHorseJ = json.loads(czHorseJ)                                            # Convert to json format

### Compile JSON

In [23]:
task_name = input('Enter a Task Name: ') # We will use 'Crazy Horse Fire' as the task name

Enter a Task Name: Crazy Horse Fire


In [ ]:
task_type = ['area']        # Our task is an area, but you could also select point
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['geotiff']             # Set output file format type
startDate = '07-01-2020'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '07-31-2020'              # End of the date range for which to extract data: MM-DD-YYYY
recurring = True                   # Specify True for a recurring date range
yearRange = [2020,2021]            # if recurring = True, set yearRange, change start/end date to MM-DD